**ВЫПОЛНИЛ ГОРОХОВ Д.Е.**

##Сбор данных средствами VK API, библиотек Selenium, BeautifulSoup, фреймворка Scrapy. 

##1. Библиотека VK API

Код, который получает список названия школ города Кемерово с помощью библиотеки vk_api и записывает результаты в файл JSON.

In [ ]:
import requests
import json
import vk_api

#database.getCities
#database.getSchools
# id Кемерово - 64
#набор и проверка работоспособности программного кода осуществлены в MS Visual Studio Community 2019.

ACCESS_TOKEN = "vk1.a.p..."
vk_session = vk_api.VkApi(token=ACCESS_TOKEN)
vk = vk_session.get_api()
_city_id = vk.database.getCities(q="Кемерово")['items'][0]['id']
result_list = vk.database.getSchools(city_id=_city_id)
with open("school.json", "w") as f:    
    result = {"Kemerovo Schools": result_list}
    json.dump(result, f, ensure_ascii=False)

##2. Запись данных в CSV формат

С помощью методов API «ВКонтакте» получаем 1000 подписчиков группы «Лентач», отсортирванных по дате регистрации.

Будут собраны следующие данные в CSV файл: пол, название город, семейное положение (ФИО партнера не указывается).

In [ ]:
import requests
import vk_api
import csv

#Использование методов
#groups.search
#users.search
#29534144 id Лентач
#набор и проверка работоспособности программного кода осуществлены в MS Visual Studio Community 2019

ACCESS_TOKEN = "vk1.a...."
vk_session = vk_api.VkApi(token=ACCESS_TOKEN)
vk = vk_session.get_api()

#groups.search
_group_id = vk.groups.search(q="Лентач")['items'][0]['id']

#users.search
result = vk.users.search(sort=1, count=1000, fields=["sex", "city", "relation" ], group_id = _group_id)
result_list = []
for item in result['items']:    
    result_list.append({
        'sex': item['sex'] if item.get('sex') else "н/д",
        'city': item['city']['title'] if item.get('city') else "city - н/д",
        'relation': item['relation'] if item.get('relation') else "relation - н/д"
        })

# Запись данных в CSV файл
with open('subscribersLentach.csv', 'w', encoding='utf-8', newline='\n') as f:
    writer = csv.DictWriter(f, fieldnames=list(result_list[0].keys()), quoting=csv.QUOTE_NONNUMERIC)
    writer.writeheader()
    for row in result_list:
        writer.writerow(row)

with open('subscribersLentach.csv', 'r', encoding='utf-8', newline='\n') as f:
    print(f.read())

##3. Скрещиваем Selenium и BeautifulSoup

Сбор информации с сайта nbcomputers.ru (https://www.nbcomputers.ru/catalog/noutbuki/) о ноутбуках данного интернет-магазина.
<br>
Собираются данные:
* Название ноутбука
* Цена ноутбука
* Код товара

Результат записывается в CSV файл.
<br>
*(P.S.: между прокликами сделаны различные временные промежутки для подгрузки страниц сайта)*

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import random
import csv

#набор и проверка работоспособности программного кода осуществлены в MS Visual Studio Community 2019

def get_html():
    service = Service(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    
    try:
        driver.get("https://www.nbcomputers.ru/catalog/noutbuki/")    
        driver.implicitly_wait(5)        
        
        actions = ActionChains(driver)    
        actions.move_to_element(driver.find_element(By.CLASS_NAME, "sc-47746e2f-0"))
        actions.perform()
            
        while True:
            wait_number = random.randint(3, 10)
            wait = WebDriverWait(driver, timeout=wait_number)    
            wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "sc-47746e2f-0"))).click()        

    except Exception as ex:
        print(f'Error: {ex}')    

    html_kod = driver.page_source
    driver.quit()
    with open("task3_html.txt", 'w', encoding='utf-8') as f:
            f.write(html_kod)

def get_csv():
    with open('task3_html.txt', 'r', encoding='utf-8', newline='\n') as f:
        html = f.read()
    soap = BeautifulSoup(html, 'lxml')
    cards_list = soap.select_one('div.fLwRJw')
    cards = cards_list.select('div.hiQXWK')
    noutbooks_info = []
    for card in cards:    
        name = card.select_one('div.buPdmH h2').text
        code = card.select_one('div.eLONJb p').text.split(" ")[1]
        price = card.select_one('div.KDZKI').text.split('₽')[0]
        noutbooks_info.append({
        'Name': name,
        'Price': price,
        'Code': code
        })
    with open('task3_noutbooksInfo.csv', 'w', encoding='utf-8', newline='\n') as f:
        writer = csv.DictWriter(f, noutbooks_info[0].keys())
        writer.writeheader()
        for row in noutbooks_info:
            writer.writerow(row)

##4. Фреймворк Scrapy

Сбор информации о заквасках с сайта pro-syr.ru (https://pro-syr.ru/zakvaski-dlya-syra/mezofilnye/)

Осуществляется сбор следующих данныех:
* Название продукта
* Цена
* Есть ли продукт в наличии

Результат записывается в CSV файл

In [ ]:
import scrapy
import time

#набор и проверка работоспособности программного кода осуществлены в MS Visual Studio Community 2019
#строка запуска файла в cmd: scrapy runspider "PythonApp_Ex4_Scrapy.py" -o "zakvaski_Ex4.csv"

class ZakvaskiSpider(scrapy.Spider):
    name = "ZakvaskiSpider"
    start_urls = ["https://pro-syr.ru/zakvaski-dlya-syra/mezofilnye/"]

    def parse(self, response):
        links = response.css("div.nameproduct a::attr(href)")
        for link in links:
            time.sleep(3)
            yield response.follow(link, self.parse_zakvaski)

        link = response.css("ul.pagination a::attr(href)")[-1].get()
        yield response.follow(link, self.parse)

    def parse_zakvaski(self, response):
        yield {
            "name": response.css("div.row h1::text").get(),
            "price": response.css("li.price h2 span.autocalc-product-price::text").get(),
            "availability": response.css("div.product-description b::text").get(),
        }

# **Дополнительно**
---



##Сбор данныех с помощью Requests

С помощью методов API ВКонтакте получаем список высших учебных заведений и названия их факультетов в городе Томск.
Результат записывается в файл JSON в следующем формате:

In [ ]:
"result": {
  "cities": [
    {   
        "id": <ID города>,
        "name": <Название города>,
        "universites": [
            {
              "id": <ID ВУЗа>, 
              "name": <Название ВУЗа>
              "faculties": [<Название факультета>, …]
            },
            ...
        ]
    },
    ...
}

In [ ]:
import requests
import json
import vk_api

#набор и проверка работоспособности программного кода осуществлены в MS Visual Studio Community 2019

ACCESS_TOKEN = "vk1.a...."
vk_session = vk_api.VkApi(token=ACCESS_TOKEN)
vk = vk_session.get_api()

faculties = []
NAME_CITY = 'Томск'
city_id = vk.database.getCities(q=NAME_CITY)['items'][0]['id']
universities = vk.database.getUniversities(city_id=city_id)['items']
university_id = [elem['id'] for elem in universities]
nameUniversities = [elem['title'] for elem in universities]
for id in university_id:
    faculties.append(vk.database.getFaculties(university_id=id)['items'])
    nameFaculties = [[f['title'] for f in fac] for fac in faculties]
result_list = [] 
for nU, uID, nF in zip(nameUniversities, university_id, nameFaculties):    
    result_list.append({
                "id": uID,
                "name": nU,
                "name faculties": nF
                })       
d = {"result": {"cities": [{"id": city_id, "name": NAME_CITY}]}}
d['result']['cities'][0]['universities']=result_list
result = []
result.append(d)
with open("tomskUniversities.json", "w") as f:    
    result = {"Tomsk Universities": result}
    json.dump(result, f, ensure_ascii=False)

##Библиотека BeautifulSoup

Код, который собирает список знаменательных дат в формате «чисто месяц год» с первой страницы сайта GCTC.ru (https://www.gctc.ru/main.php?id=98.1)

In [ ]:
import requests
from bs4 import BeautifulSoup
import lxml
import csv

#набор и проверка работоспособности программного кода осуществлены в MS Visual Studio Community 2019

URL = "https://www.gctc.ru/main.php?id=98.{}"

session = requests.session()
session.headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'Accept-Language': 'ru,ru-RU;q=0.9,en-US;q=0.8,en;q=0.7'}

months = {"января":"январь", "февраля":"февраль", "марта":"март","апреля":"апрель","мая":"май","июня":"июнь","июля":"июль","августа":"август","сентября":"сентябрь","октября":"октябрь","ноября":"ноябрь","декабря":"декабрь"}
result = []
for page in range(1,13):
    res = session.get(URL.format(page))
    soap = BeautifulSoup(res.text, 'lxml')  
    informs = soap.select_one('div.ie_infoh')
    dates = informs.select('h1.shad')
    years = informs.select('div.news')

    for date, year in zip(dates, years):
        day = date.text[:2]
        mm = months[date.select_one('span.s12').text]    
        yy = year.select_one('h2').text.split()[0]
        even = year.text.split('\n')[1]
        result.append({
            'число':day,
            'месяц': mm,
            'год': yy,
            'событие': even
            })

with open('addTask2.csv', 'w', encoding="utf-8", newline='\n') as f:
  writer = csv.DictWriter(f, result[0].keys())
  writer.writeheader()
  for row in result:
    writer.writerow(row)